In [ ]:
#What is the problem??
#It dramatically depends on the kind of fantasy league you are playing
#(1)Trying to predict which player combinations will win you the most number of points in any given week?
#(2)Which players are going to be the best players next week/year/month?
#One thing that is interesting is that best players probably, but might not, add up to the best teams



In [14]:
import numpy as np
import pandas as pd
import requests
import keras
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import random
import calendar

%matplotlib inline  


In [ ]:
#http://danielfrg.com/blog/2013/04/01/nba-scraping-data/
url = 'http://espn.go.com/nba/teams'
r = requests.get(url)

soup = BeautifulSoup(r.text)
tables = soup.find_all('ul', class_='medium-logos')

teams = []
prefix_1 = []
prefix_2 = []
teams_urls = []
for table in tables:
    lis = table.find_all('li')
    for li in lis:
        info = li.h5.a
        teams.append(info.text)
        url = info['href']
        teams_urls.append(url)
        prefix_1.append(url.split('/')[-2])
        prefix_2.append(url.split('/')[-1])


dic = {'url': teams_urls, 'prefix_2': prefix_2, 'prefix_1': prefix_1}
teams = pd.DataFrame(dic, index=teams)

In [ ]:
team_prefixes_bbal_ref = ['BOS', 'BRK', 'NYK', 'PHI', 'TOR', 'GSW', 'LAC', 'LAL', 'PHO', 'SAC', 
                          'CHI', 'CLE', 'DET', 'IND', 'MIL', 'DAL', 'HOU', 'MEM','SAS', 'ATL','CHO', 'MIA',
                         ]

In [ ]:
BASE_URL_BBALL_REF = 'http://www.basketball-reference.com/teams/CLE/2017.html'
r = requests.get(BASE_URL_BBALL_REF)
table = BeautifulSoup(r.text)

In [ ]:
BASE_URL_BBALL_REF = 'http://www.basketball-reference.com/teams/{0}/{1}.html'
r = requests.get(BASE_URL_BBALL_REF)
column_headers = [u'Name', u'Age', u'G', u'GS', u'MP', u'FG', u'FGA', u'FG%', u'3P', u'3PA', u'3P%', u'2P', u'2PA', u'2P%', u'eFG%', u'FT', u'FTA', u'FT%', u'ORB', u'DRB', u'TRB', u'AST', u'STL', u'BLK', u'TOV', u'PF', u'PTS']

for year in range(1995, 2018):
    player_data = []
    for index, row in teams.iterrows():
        r = requests.get(BASE_URL_BBALL_REF.format(row['prefix_1'].upper(), year))
        html = str(BeautifulSoup(r.text).find(id="all_totals"))
        formatted_html = html.replace("<!--", "")
        table = BeautifulSoup(formatted_html).find(id="totals").find_all('tr')
        player_data_per_team = []
        for row in table[1:]:
            data = [td.getText() for td in row.findAll('td')]
            player_data_per_team.append(data)
        without_totals = player_data_per_team[:-1]
        print(without_totals)
        #formatted_data = [[data[0].encode('utf-8')] + [float(x) for x in data[1:]] for data in without_totals]
        player_data.append(without_totals)
    df = pd.DataFrame(player_data, columns=column_headers)
    df.to_csv("player_stats_bball_ref" + str(year) + ".csv")

In [ ]:
html = str(BeautifulSoup(r.text).find(id="all_totals"))
formatted = html.replace("<!--", "")
table = BeautifulSoup(formatted).find(id="totals").find_all('tr')

headers = [u'Name', u'Age', u'G', u'GS', u'MP', u'FG', u'FGA', u'FG%', u'3P', u'3PA', u'3P%', u'2P', u'2PA', u'2P%', u'eFG%', u'FT', u'FTA', u'FT%', u'ORB', u'DRB', u'TRB', u'AST', u'STL', u'BLK', u'TOV', u'PF', u'PTS']

player_data = []
for row in table[1:]: # Remove header
    data = [td.getText() for td in row.findAll('td')]
    player_data.append(data)
data_without_totals = player_data[:-1]
formatted_data = [[data[0].encode('utf-8')] + [float(x) for x in data[1:]] for data in data_without_totals]
pd.DataFrame(formatted_data, columns = headers)
        #player_data = []
#for row in table: # Remove header
 #   print(row.findAll('td').length)
  #  data = [x.getText() for x in row.findAll('td')]
   # formatted_data = [data[0].encode('utf-8')] + [float(x) for x in data[1:]]
    #player_data.append(formattedData)

# Collect data from espn from 2003 to 2017

In [ ]:
BASE_URL = 'http://www.espn.com/nba/team/stats/_/name/{0}/year/{1}'

column_headers = [u'PLAYER', u'GP', u'GS', u'MIN', u'PPG', u'OFFR', u'DEFR', u'RPG', u'APG', u'SPG', u'BPG', u'TPG', u'FPG', u'A/TO', u'PER']

def normalizeColumn(column):
    theMax = column.max()
    theMin = column.min()
    normalizedColumn = [ (record - theMin) / (theMax - theMin) for record in column]
    return normalizedColumn

for year in range(2003, 2018):
    player_data = []
    for index, row in teams.iterrows():
        r = requests.get(BASE_URL.format(row['prefix_1'], year))
        table = BeautifulSoup(r.text).table
        for row in table.find_all('tr')[2:][:-1]: # Remove header
            data = [td.getText() for td in row.findAll('td')]
            formattedData = [data[0].encode('utf-8')] + [float(x) for x in data[1:]] #formats the none-name fields to floats so we can do math
            player_data.append(formattedData)
    df = pd.DataFrame(player_data, columns=column_headers)
    for column in df.iloc[:,1:]:
        data = normalizeColumn(df[column])
        df[column + "_Normalized"] = data
    df['scores_Normalized'] = df.iloc[:,15:].apply(sum, axis = 1)
    df.to_csv("player_stats_" + str(year) + ".csv")

# Normalize monster basketball data

In [ ]:
def normalizeColumn(column):
    theMax = column.max()
    theMin = column.min()
    normalizedColumn = [ (record - theMin) / float((theMax - theMin)) for record in column]
    return normalizedColumn

categories = ['p','a','3','r','s','b','to', 'fg%','ft%','fta']
normed_categores = [cat + "_Normalized" for cat in categories]
dates = range(2004, 2018)
test = None
for date in dates:
    _file = 'monster_players_with_ranks_{0}.xls'.format(date)
    df = pd.read_excel(_file)
    for x in categories:
        cat = df[x]
        normed = normalizeColumn(cat)
        df[x + "_Normalized"] = normed
    scores = [sum([row[cat] for cat in normed_categores]) for label,row in df.iterrows()]
    df['normalized_score'] = scores
    df.to_csv("normalized_" + _file)

In [ ]:
pd.read_csv('normalized_monster_players_with_ranks_2013.xls')

In [300]:
plt.scatter(x=iters, y=mse2)

NameError: name 'mse2' is not defined

# Get all the games for all years between 2005 and 2017

In [706]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, date

teams = pd.read_csv('teams.csv')
BASE_URL = 'http://espn.go.com/nba/team/schedule/_/name/{0}/year/{1}/{2}'

for year in range(2005, 2018):
    match_id = []
    dates = []
    home_team = []
    home_team_score = []
    visit_team = []
    visit_team_score = []
    for index, row in teams.iterrows():
        _team, url = row['team'], row['url']
        r = requests.get(BASE_URL.format(row['prefix_1'], year, row['prefix_2']))
        table = BeautifulSoup(r.text).table
        for row in table.find_all('tr')[1:]: # Remove header
            columns = row.find_all('td')
            try:
                _home = True if columns[1].li.text == 'vs' else False
                _other_team = columns[1].find_all('a')[1].text
                _score = columns[2].a.text.split(' ')[0].split('-')
                _won = True if columns[2].span.text == 'W' else False
                _match_id = columns[2].a['href'].split('id/')[1]
                d = datetime.strptime(columns[0].text, '%a, %b %d')
                print(columns)
                raise Exception('I know Python!')
                dates.append(date(year, d.month, d.day))
                match_id.append(_match_id)
                home_team.append(_team if _home else _other_team)
                visit_team.append(_team if not _home else _other_team)
                
                if _home:
                    if _won:
                        home_team_score.append(_score[0])
                        visit_team_score.append(_score[1])
                    else:
                        home_team_score.append(_score[1])
                        visit_team_score.append(_score[0])
                else:
                    if _won:
                        home_team_score.append(_score[1])
                        visit_team_score.append(_score[0])
                    else:
                        home_team_score.append(_score[0])
                        visit_team_score.append(_score[1])
            except Exception as e:
                pass # Not all columns row are a match, is OK
                # print(e)

    dic = {'id': match_id, 'date': dates, 'home_team': home_team, 'visit_team': visit_team,
            'home_team_score': home_team_score, 'visit_team_score': visit_team_score}

    games = pd.DataFrame(dic).drop_duplicates().set_index('id')
    games.to_csv('games_' + str(year)+"_v2.csv")

[<td>Wed, Nov 3</td>, <td><ul class="game-schedule"><li class="game-status">vs</li><li class="team-logo-small logo-nba-small"><a href="http://www.espn.com/nba/team/_/name/phi/philadelphia-76ers"><img src="http://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/Phi.png&amp;h=80&amp;w=80"/></a></li><li class="team-name"><a href="http://www.espn.com/nba/team/_/name/phi/philadelphia-76ers">Philadelphia</a></li></ul></td>, <td><ul class="game-schedule"><li class="game-status loss"><span class="redfont">L</span></li><li class="score"><a href="//www.espn.com/nba/recap/_/id/241103002">98-95</a></li></ul></td>, <td>0-1</td>, <td colspan="1"><a href="http://www.espn.com/nba/player/_/id/662/paul-pierce">P. Pierce</a> 35</td>, <td colspan="1"><a href="http://www.espn.com/nba/player/_/id/662/paul-pierce">P. Pierce</a> 13</td>, <td colspan="1"><a href="http://www.espn.com/nba/player/_/id/640/gary-payton">G. Payton</a> 8</td>]
[<td>Fri, Nov 5</td>, <td><ul class="game-schedule"><li class=

KeyboardInterrupt: 

In [4]:
pd.read_csv('games_2014.csv').drop_duplicates('id')

,id,date,home_team,home_team_score,visit_team,visit_team_score
0,400488879,2014-10-30,Toronto,93,Boston Celtics,87
1,400488897,2014-11-01,Boston Celtics,98,Milwaukee,105
2,400488914,2014-11-03,Detroit,87,Boston Celtics,77
3,400488922,2014-11-04,Memphis,95,Boston Celtics,88
4,400488937,2014-11-06,Boston Celtics,97,Utah,87
5,400488948,2014-11-08,Orlando,89,Boston Celtics,91
6,400488962,2014-11-09,Miami,110,Boston Celtics,111
7,400488974,2014-11-11,Boston Celtics,120,Orlando,105
8,400488986,2014-11-13,Boston Celtics,83,Charlotte,89
9,400489001,2014-11-15,Boston Celtics,96,Portland,109


In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, date

games = pd.read_csv('games_2014.csv').drop_duplicates('id').set_index('id')
BASE_URL = 'http://espn.go.com/nba/boxscore?gameId={0}'
columns = ['id', 'player', u'MIN', u'FG', u'3PT', u'FT', u'REB', u'AST', u'STL', u'BLK', u'TO', u'PF', u'PTS']

players = []

def get_players(players, team_name):
    array = np.zeros((len(players), len(headers)+1), dtype=object)
    array[:] = np.nan
    for i, player in enumerate(players):
        cols = player.find_all('td')
        array[i, 0] = cols[0].text.split(',')[0]
        for j in range(1, len(headers) + 1):
            if not cols[1].text.startswith('DNP'):
                array[i, j] = cols[j].text

    frame = pd.DataFrame(columns=columns)
    for x in array:
        line = np.concatenate(([index, team_name], x)).reshape(1,len(columns))
        new = pd.DataFrame(line, columns=frame.columns)
        frame = frame.append(new)
    return frame

for index, row in games.iterrows():
    print(index)
    request = requests.get(BASE_URL.format(index))
    html = BeautifulSoup(request.text)
    table = html.find('table', class_='mod-data')
    for c in range (1,5):
        for x in BeautifulSoup(request.text).find_all('tbody')[c].find_all('tr'):
            try:
                position = x.find(class_="position").getText()
                name = x.find(class_="abbr").getText()
                _min = x.find(class_="min").getText()
                threePt = x.find(class_="3pt").getText()
                ft = x.find(class_="ft").getText()
                fg = x.find(class_="fg").getText()
                reb = x.find(class_="reb").getText()
                ast = x.find(class_="ast").getText()        
                stl = x.find(class_="stl").getText() 
                blk = x.find(class_="blk").getText()
                to = x.find(class_="to").getText() 
                pf = x.find(class_="pf").getText()
                pts = x.find(class_="pts").getText()
                data = [index, name, _min, fg, ft, threePt, reb, ast, stl, blk, to, pf, pts]
                players.append(data)
                print(players.shape)
            except Exception as e:
                    pass
stats_2014 = pd.DataFrame(data = players, columns=columns)

400488879


/Library/Python/2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 162 of the file /System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


400488897
400488914
400488922
400488937
400488948
400488962
400488974
400488986
400489001
400489015
400489032
400489043
400489052
400489063
400489075
400489090
400489105
400489120
400489135
400489156
400489172
400489187
400489192
400489206
400489231
400489246
400489266
400489277
400489309
400489333
400489348
400489357
400489375
400489390
400489401
400489414
400489424
400489429
400489442
400489459
400489475
400489490
400489498
400489509
400489531
400489543
400489551
400489582
400489599
400489616
400489631
400489640
400489653
400489680
400489697
400489704
400489718
400489727
400489750
400489780
400489793
400489810
400489823
400489835
400489848
400489863
400489874
400489882
400489899
400489936
400489949
400489970
400489979
400489992
400490006
400490019
400490039
400490051
400490066
400490077
400490095
400488877
400488901
400488916
400488925
400488950
400488960
400488994
400489005
400489019
400489023
400489034
400489055
400489070
400489084
400489091
400489107
400489116
400489136
400489150


In [10]:
def convert(x):
    try:
        return float(x)
    except Exception as e:
        return 0.0

In [4]:
modified = stats_2014.drop([14204, 14208, 24541])
ts = modified['3PT']
t = np.array([[convert(x) for x in threes.split('-')] for threes in ts])

In [5]:
modified['3PTsMade'] = t.T[0]

In [6]:
modified['3PTsAttempted'] = t.T[1]

In [7]:
ts = modified['FT']
t = np.array([[convert(x) for x in threes.split('-')] for threes in ts])
modified['FTsMade'] = t.T[0]
modified['FTsAttempted'] = t.T[1]
ts = modified['FG']
t = np.array([[convert(x) for x in threes.split('-')] for threes in ts])
modified['FGsMade'] = t.T[0]
modified['FGsAttempted'] = t.T[1]
stats_2014 = modified


In [9]:
stats_2014.to_csv('stats_2014_player_stats_by_game')